In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import string
import numpy as np
import random
import logging
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.INFO)

# Parameters
BATCH_SIZE = 100
IMG_SIZE = (128, 64)  # Resizing the image
EPOCHS = 200
LEARNING_RATE = 0.001
NUM_LABELS = 100  # Pick 100 random labels

# Load dataset filenames
dataset_path = '/kaggle/input/precogtask-easy/easy_captcha_dataset'
all_files = [f for f in os.listdir(dataset_path) if f.endswith('.png')]
random.shuffle(all_files)
selected_files = all_files[:NUM_LABELS]  # Pick 100 random images
label_to_index = {f.split('_')[1]: i for i, f in enumerate(selected_files)}


# Data Loading

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def encode_label(text):
    """Convert text label to a numerical class index."""
    return torch.tensor(label_to_index[text], dtype=torch.long, device=device)
class CaptchaDataset(Dataset):
    def __init__(self, root, files, transform=None):
        self.root = root
        self.files = files
        self.transform = transform
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        img_name = self.files[idx]
        label_text = img_name.split('_')[1]  # Extract text from filename
        image = Image.open(os.path.join(self.root, img_name)).convert('L')  # Convert to grayscale
        if self.transform:
            image = self.transform(image)
        label = encode_label(label_text)
        return image.to(device), label


In [30]:
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])
dataset = CaptchaDataset(dataset_path, selected_files, transform=transform)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [31]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        with torch.no_grad():
            sample_input = torch.zeros(1, 1, IMG_SIZE[1], IMG_SIZE[0])  # (batch, channels, height, width)
            sample_output = self.conv_layers(sample_input)
            self.flattened_size = sample_output.view(1, -1).shape[1]
        
        self.fc = nn.Sequential(
            nn.Linear(self.flattened_size, 512),
            nn.ReLU(),
            nn.Linear(512, NUM_LABELS)  # Predict one of 100 labels
        )
        
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x  # Output is a class index prediction

# Initialize Model

In [32]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Initialize lists for tracking evolution
epoch_losses = []
epoch_accuracies = []

# Training Loop
for epoch in range(EPOCHS):
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)  # CrossEntropyLoss expects class indices
        loss.backward()
        optimizer.step()
        
        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        # Accumulate predictions and true labels
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
    accuracy = 100 * correct / total
    epoch_losses.append(loss.item())
    epoch_accuracies.append(accuracy)
    
    # Compute metrics
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    
    logging.info(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%, Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")
    logging.info(f"Confusion Matrix:\n{cm}")

# Plot evolution of loss and accuracy
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(epoch_losses, label='Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Evolution')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epoch_accuracies, label='Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy Evolution')
plt.legend()

plt.show()


Epoch [1/200], Loss: 4.6056, Accuracy: 1.00%
Epoch [2/200], Loss: 4.6295, Accuracy: 1.00%
Epoch [3/200], Loss: 4.6063, Accuracy: 1.00%
Epoch [4/200], Loss: 4.6061, Accuracy: 1.00%
Epoch [5/200], Loss: 4.6053, Accuracy: 1.00%
Epoch [6/200], Loss: 4.6048, Accuracy: 1.00%
Epoch [7/200], Loss: 4.6044, Accuracy: 1.00%
Epoch [8/200], Loss: 4.6040, Accuracy: 1.00%
Epoch [9/200], Loss: 4.6035, Accuracy: 1.00%
Epoch [10/200], Loss: 4.6028, Accuracy: 1.00%
Epoch [11/200], Loss: 4.6017, Accuracy: 2.00%
Epoch [12/200], Loss: 4.6001, Accuracy: 2.00%
Epoch [13/200], Loss: 4.5978, Accuracy: 3.00%
Epoch [14/200], Loss: 4.5946, Accuracy: 5.00%
Epoch [15/200], Loss: 4.5902, Accuracy: 6.00%
Epoch [16/200], Loss: 4.5841, Accuracy: 5.00%
Epoch [17/200], Loss: 4.5746, Accuracy: 2.00%
Epoch [18/200], Loss: 4.5610, Accuracy: 3.00%
Epoch [19/200], Loss: 4.5417, Accuracy: 4.00%
Epoch [20/200], Loss: 4.5147, Accuracy: 4.00%
Epoch [21/200], Loss: 4.4765, Accuracy: 5.00%
Epoch [22/200], Loss: 4.4281, Accuracy: 4.0

In [ ]:
# NEW CELL: Example Predictions
model.eval()
with torch.no_grad():
    sample_images, sample_labels = next(iter(dataloader))
    sample_images = sample_images.to(device)
    outputs = model(sample_images)
    _, predicted = torch.max(outputs, 1)
    # Invert the label_to_index mapping
    index_to_label = {v: k for k, v in label_to_index.items()}
    for i in range(min(5, sample_images.size(0))):
        true_label = index_to_label[sample_labels[i].item()]
        pred_label = index_to_label.get(predicted[i].item(), "Unknown")
        print(f"Sample {i+1}: True Label: {true_label} | Predicted: {pred_label}")
